# Isogrid Passthrough Design for the LV4 Sub-Orbital Rocket
## Motivation
Since we are moving to a fully machined aluminum airframe, the passthrough rings are another potential part for optimization with isogrid design. Hand calcs derived from NASA Isogrid Handbook section 4.8

In [1]:
# Stock properties
grid_id = 12.5     # in
grid_wall_t = 0.25  # in
grid_od = grid_id+2*grid_wall_t # in
grid_r = grid_od/2   # in
grid_length = 4  # in
offset = 0.05

# Material properties for Al 6061-T6
E_Si = 69.0 # Young's Modules, GPa
density = 0.0975 #lb/in^3
E = 10.1e6 # psi
S_y = 34e3 # Yield compressive Strength - psi
S_u = 45e3 # Ultimate Tensile Strength - psi

nu = 0.33 # Poisson's Ratio


### Safety Factor
Minimum safety factor for Base 11 is 1.5 (see requirement 3.2.4.15)

In [2]:
safety_factor = 1.5


### Pattern size
For optimal strength and to maintain isotropic properties, the isogrid cells need to wrap around the tank diameter in integer increments. Twenty four cells was chosen as a starting point.

In [3]:
import math
num_radial_divisions = 12
pattern_size = grid_od*math.pi/num_radial_divisions
pattern_height = pattern_size*(3**(1/2))/2
print('Pattern Size: {:5.5f}'.format(pattern_size))
print('Pattern Height: {:5.5f}'.format(pattern_height))


Pattern Size: 3.40339
Pattern Height: 2.94742


### Design Properties

from "Isogrid Definitions", page viii

**Design choices:**<br>
b: rib width; &emsp;&emsp;&emsp;&emsp;  min 0.03" per Peter<br>
d: rib depth; &emsp;&emsp;&emsp;&emsp; calculated from OD minus skin<br>
c: flange depth<br>
w: flange width<br>
s: plate thickness of unflanged isogrid<br>
h: triangle height<br>
a: triangle leg length (center to center along pattern)<br><br>

**Equivalent material properties:**<br>
t_eff = equivalent thickness for membrane stresses<br>
t_bar = equivalent weight thickness <br>
E_star = equivalent monocoque Young's modulus<br><br>

In [4]:
#Design choices
b = 0.05
d = grid_wall_t-offset
c = 0.0
w = 0.0
h = pattern_height
a = pattern_size


# non dimensional terms α λ μ δ β
β=b/a

#Equivalent properties
E_bar = E*b/h
t_bar = 2*(3**0.5)*β*d #4.8.16
t_eff = b*d/h


## Mass Model
Equations for calculating the mass of the isotank based on diameter and length

tbar: equiv wt thickness<br>

In [5]:
import math

cross_section = math.pi*((grid_id/2+t_bar)**2-(grid_id/2)**2)
volume = cross_section*grid_length
grid_mass = volume*density
lbm_to_kg = 0.453592 #unit conversion
grid_mass_kg = grid_mass * lbm_to_kg

print('equivalent weight thickness:',t_bar, 'in')
print('grid_mass:',grid_mass, 'lbm')
print('grid_mass_kg:',grid_mass_kg, 'kg')
#print('tank_wall_mass:',tank_wall_mass, 'kg')

equivalent weight thickness: 0.010178379607786931 in
grid_mass: 0.15601150486965917 lbm
grid_mass_kg: 0.07076557051683845 kg


## Load Calculations



### Combined Compression and Bending
For the fuel tank, the cylinder is under compression and bending. Section 4.2 on page 4.2.001 (#97) covers this scenario. The example covers:
* General Instability (N_cr_1)
* Skin Buckling (N_cr_2)
* Rib Crippling (N_cr_3)

In [6]:
import math

# Inputs
F = -1573 # 1573 lbf ~7000 N
M = 265 # 265 lbf*in = 30 Nm

N_cr_1 = 0.460*E*β*d**2/grid_r
N_cr_2 = 1.422*E*d**3

#print('If Ncr_Nb_check > Nb: \n edit Euler modifier ({:1.2f}) using ref 2.1 (4.2.002) \n https://sci-hub.se/https://doi.org/10.1017/S0001924000014755'.format(euler))
#print('Euler buckling lookup values:\n R/t*: {:5.1f} \n L/R: {:5.1f}'.format(R_t_star,L_R))

#print('\nBuckling: \n classical (Nb > check value above): \n  {:5.1f} lbf/in || {:5.1f} lbf'.format(N_cl,N_cl*2*math.pi*grid_r))
#print(' conservative (Euler): \n  {:5.1f} lbf/in || {:5.1f} lbf'.format(N_cr_timo,N_cr_timo*2*math.pi*grid_r))
#print(' calculated (KDF = {:1.2f}) \n  ref: https://www.hindawi.com/journals/ijae/2020/9851984/#B4\n  {:5.1f} lbf/in || {:5.1f} lbf'.format(Y,N_cr_KDL,N_cr_KDL*2*math.pi*grid_r))
print('\nGeneral Instability: \n {:5.1f} lbf/in || {:5.1f} lbf'.format(N_cr_1,N_cr_1*2*math.pi*grid_r))
#print('General Instability (conservative KDF = {:1.2f} from 4.2.005): \n {:5.1f} lbf/in || {:5.1f} lbf'.format(Y_con,N_cr_1_con,N_cr_1_con*2*math.pi*grid_r))
### I realized that General Instability and Buckling are the same thing; not sure which of these apply, probably best to look at the most conservative or analyze in FEA.

#print('K_c: \n {:5.1f} lbf'.format(K_c))

print('Rib Crippling: \n {:5.1f} lbf/in || {:5.1f} lbf'.format(N_cr_2,N_cr_2*2*math.pi*grid_r))




General Instability: 
 420.0 lbf/in || 17154.5 lbf
Rib Crippling: 
 114897.6 lbf/in || 4692498.9 lbf


## Stress Calcs

In [7]:
Fx = F #axial
Fy = 0 #circumferential
Txy = 0.0

Na = F/(math.pi*grid_id)
Nb = M/(math.pi*(grid_id/2)**2)
N_max = Na+Nb
N_min = Na-Nb

σ_1_rib = -(h/(3*b*d))*N_max
σ_2_3_rib = (2*h/(3*b*d))*N_max


#add in membrane rib and skin stresses (2.0.021)

print('Loads:\n axial:\n  {:5.1f} lbf/in || {:5.1f} lbf'.format(Na,Na*2*math.pi*grid_r))
print(' bending:\n  {:5.1f} lbf/in || {:5.1f} lbf'.format(Nb,Nb*2*math.pi*grid_r))
print(' combined max:\n  {:5.1f} lbf/in || {:5.1f} lbf'.format(N_max,N_max*math.pi*grid_id))
print(' combined min:\n  {:5.1f} lbf/in || {:5.1f} lbf'.format(N_min,N_min*math.pi*grid_id))
print('\nMembrane rib stress:')
print(' σ_1 rib: {:5.1f} psi'.format(σ_1_rib))
print(' σ_2_3 rib: {:5.1f} psi'.format(σ_2_3_rib))

Loads:
 axial:
  -40.1 lbf/in || -1635.9 lbf
 bending:
    2.2 lbf/in ||  88.2 lbf
 combined max:
  -37.9 lbf/in || -1488.2 lbf
 combined min:
  -42.2 lbf/in || -1657.8 lbf

Membrane rib stress:
 σ_1 rib: 3723.3 psi
 σ_2_3 rib: -7446.5 psi
